# Adicionando as ferramentas ao modelo

A última etapa é realmente chamarmos essas tools quando o modelo solicitar, criando assim uma aplicação funcional que combina modelo com ferramentas criadas pelo usuário.

## Recriando as tools

Vamos primeiro recriar as tools da última aula.

In [1]:
from dotenv import load_dotenv, find_dotenv
_=load_dotenv(find_dotenv())

In [ ]:
import requests
import datetime
from langchain.agents import tool
from pydantic import BaseModel, Field

class RetornaTempArgs(BaseModel):
    latitude: float = Field(description='Latitude da localidade que buscamos a temperatura')
    longitude: float = Field(description='Longitude da localidade que buscamos a temperatura')

@tool(args_schema=RetornaTempArgs)
def retorna_temperatura_atual(latitude: float, longitude: float):
    """Retorna a temperatura atual dada uma determinada coordenada"""
    URL = 'https://api.open-meteo.com/v1/forecast'

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    resposta = requests.get(URL, params=params)
    if resposta.status_code == 200:
        resultado = resposta.json()
        hora_agora = datetime.datetime.now(datetime.UTC).replace(tzinfo=None)
        lista_horas = [datetime.datetime.fromisoformat(temp_str) for temp_str in resultado['hourly']['time']]
        index_mais_prox = min(range(len(lista_horas)), key=lambda x: abs(lista_horas[x] - hora_agora))
        temp_atual = resultado['hourly']['temperature_2m'][index_mais_prox]
        return temp_atual
    else:
        raise Exception(f'Request para a API {URL} falhou: {resposta.status_code}')

In [3]:
import wikipedia
wikipedia.set_lang('pt')

@tool
def busca_wikipedia(query: str):
    """Faz uma busca na Wikipédia e retorna resumos de páginas para a query"""
    titulos_paginas = wikipedia.search(query)
    resumos = []
    for titulo in titulos_paginas[:3]:
        try:
            wiki_page = wikipedia.page(title=titulo, auto_suggest=True)
            resumos.append(f"Título da página: {titulo}\nResumo: {wiki_page.summary}")
        except:
            pass
    if not resumos:
        return "Busca não teve retorno"
    else:
        return '\n\n'.join(resumos)

In [24]:
from langchain.prompts import ChatPromptTemplate as cpt
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function

prompt = cpt.from_messages([
    ('system', 'Você é um assitente amigável chamado Isaac'),
    ('user', '{input}')
])

chat = ChatOpenAI(model='gpt-4o-mini')

tools = [busca_wikipedia, retorna_temperatura_atual]
tools_json = [convert_to_openai_function(tool) for tool in tools]
tool_run = {tool.name: tool for tool in tools}

chain = prompt | chat.bind(functions=tools_json)

In [25]:
tool_run

{'busca_wikipedia': StructuredTool(name='busca_wikipedia', description='Faz uma busca na Wikipédia e retorna resumos de páginas para a query', args_schema=<class 'langchain_core.utils.pydantic.busca_wikipedia'>, func=<function busca_wikipedia at 0x000001FF3AA55260>),
 'retorna_temperatura_atual': StructuredTool(name='retorna_temperatura_atual', description='Retorna a temperatura atual dada uma determinada coordenada', args_schema=<class 'langchain_core.utils.pydantic.retorna_temperatura_atual'>, func=<function retorna_temperatura_atual at 0x000001FF3AA55120>)}

## Criando roteamento para rodar ferramenta

Chamamos de roteamento o processo de análise da output de um modelos que possui acesso a ferramntas. Noste processo, precisamos entender qual é o tipo de saída que o modelo está nos fornecendo. Caso seja uma saída já com a resposta final do modelo, podemos retorná-la ao usuário, caso o modelo esteja solicitando a chamado de uma ferramenta, temos que chamar essa ferramenta e mostrar o resultado ao usuário.

### Adicionando OpenAIFunctionsAgentOutputParser

Para auxiliar neste processo, podemos utilizar o OpenAIFunctionsAgentOutputParser. Ele processa a saída de um modelo da OpenAi que possui ferramentas e determina o estado da mensagem devolvida.

In [26]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser()

O retorno será um AgentAction caso necessite que uma ferramenta seja executada.

In [27]:
resultado = chain.invoke({'input':'O que é God of War?'})
resultado

AgentActionMessageLog(tool='busca_wikipedia', tool_input={'query': 'God of War'}, log="\nInvoking: `busca_wikipedia` with `{'query': 'God of War'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"God of War"}', 'name': 'busca_wikipedia'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 104, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'function_call', 'logprobs': None}, id='run-88005209-bf0d-43cd-b82d-cbe4fba04d4a-0', usage_metadata={'input_tokens': 104, 'output_tokens': 18, 'total_tokens': 122, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})])

In [28]:
resultado.tool

'busca_wikipedia'

E um AgentFinish, caso a mensagem esteja finalizada.

In [29]:
chain.invoke({'input':'Olá'})

AgentFinish(return_values={'output': 'Olá! Como posso te ajudar hoje?'}, log='Olá! Como posso te ajudar hoje?')

### Rodando as ferramentas

Podemos criar uma função simples de roteamento para lidar com os dois estados possíveis:

In [30]:
from langchain.agents.agent import AgentFinish

def roteamento(resultado):
    if isinstance(resultado, AgentFinish):
        return resultado.return_values['output']
    else:
        return tool_run[resultado.tool].run(resultado.tool_input)

In [33]:
chain = prompt | chat.bind(functions=tools_json) | OpenAIFunctionsAgentOutputParser() | roteamento

In [34]:
chain.invoke({'input': 'Olá'})

'Olá! Como posso ajudar você hoje?'

In [35]:
chain.invoke({'input': 'Quem foi Isaac Asimov?'})

'Título da página: Isaac Asimov\nResumo: Isaac Asimov (em russo: Исаак Юдович Озимов; romaniz.: Isaak Yudavich Azimov; Petrovichi, Rússia Soviética, atual Rússia, 2 de janeiro de 1920 — Brooklyn, 6 de abril de 1992) foi um escritor e bioquímico russo-americano, autor de obras de ficção científica e divulgação científica.\nAsimov é considerado um dos mestres da ficção científica e, junto com Robert A. Heinlein e Arthur C. Clarke, foi considerado um dos "três grandes" dessa área da literatura. A obra mais famosa de Asimov é a Série da Fundação, também conhecida como Trilogia da Fundação, que faz parte da série do Império Galáctico e que logo combinou com a Série Robôs. Também escreveu obras de mistério e fantasia, assim como uma grande quantidade de não-ficção. No total, escreveu ou editou mais de 500 volumes, aproximadamente 90 000 cartas ou postais, e tem obras em cada categoria importante do sistema de classificação bibliográfica de Dewey, exceto em filosofia.\nA maioria de seus livro